In [1]:
import darkwing as dw

def foo(rel, res=6):
    return (rel
    | 'select pickup_latitude as lat, pickup_longitude as lng, tip_amount'       
    | f'select h3_latlng_to_cell(lat, lng, {res}) as hexid, tip_amount as tip'
    | 'select hexid, avg(tip) as tip group by 1'
    | 'select h3_h3_to_string(hexid) as hexid, tip'
    | 'where tip > 0'
    )

In [2]:
rel1 = dw.load('data/yellow_tripdata_2010-01.parquet') # lazy load!
rel2 = dw.load('data/yellow_tripdata_2010-02.parquet') # lazy load!

In [3]:
# TODO: why isn't this lazy?
# rel3 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-01.parquet' >> dw.load

In [4]:
rel1 | foo

┌─────────────────┬────────────────────┐
│      hexid      │        tip         │
│     varchar     │       double       │
├─────────────────┼────────────────────┤
│ 862a1001fffffff │ 0.7062534435261708 │
│ 862a106d7ffffff │ 0.6087299035369775 │
│ 862a1018fffffff │ 1.6588950276243093 │
│ 862a1088fffffff │  9.787446808510639 │
│ 862a10667ffffff │ 0.6512962962962964 │
│ 862a10707ffffff │ 1.4344105263157896 │
│ 862a1071fffffff │  6.421379129522809 │
│ 862a1000fffffff │ 1.2942699115044247 │
│ 862a15667ffffff │ 3.6666666666666665 │
│ 862a10187ffffff │  4.194242424242424 │
│        ·        │                 ·  │
│        ·        │                 ·  │
│        ·        │                 ·  │
│ 862a1515fffffff │               3.56 │
│ 862b8a727ffffff │                1.0 │
│ 862b8b387ffffff │               9.95 │
│ 862aa314fffffff │                1.7 │
│ 862a1385fffffff │                2.0 │
│ 862a1546fffffff │                1.0 │
│ 862b8a6e7ffffff │                1.0 │
│ 862a16717fffff

In [5]:
out = dw.Databoose(
    rel1 = rel1 >> foo,
    rel2 = rel2 >> foo,
) | """
select
    hexid,
    rel1.tip
        as tip1,
    rel2.tip
        as tip2,
from rel1
full outer join rel2
using (hexid)
"""

(out
| 'where tip1 is not null and tip2 is not null'
| 'select *, tip1-tip2 as diff order by diff'
| 'as bam'
)

Databoose:
    bam: 371 x ['hexid', 'tip1', 'tip2', 'diff']